In [2]:
import numpy as np
from skimage import io
from pathlib import Path
import re
#import ants
from skimage.transform import resize
from tqdm import tqdm
from skimage.morphology import skeletonize_3d, binary_closing
from scipy.ndimage import distance_transform_edt, binary_dilation
import tifffile as tif
from scipy.ndimage import binary_fill_holes
import cc3d
from scipy.io import loadmat, savemat
import sknw
import networkx as nx
import pickle
import os
import matplotlib.pyplot as plt
import pandas as pd
import time
import scipy as sp
import vg
from pytransform3d.rotations import matrix_from_axis_angle
import multiprocessing
from scipy.ndimage import convolve as conv
from scipy.stats import multivariate_normal
from skimage import color, data, restoration
from RedLionfishDeconv import doRLDeconvolutionFromNpArrays
from matplotlib.patches import Circle
from skimage.feature import peak_local_max
from statistics import mode
import imageio
from PIL import Image
from PIL.TiffTags import TAGS
from tifffile import TiffFile
from sklearn.metrics import normalized_mutual_info_score, adjusted_mutual_info_score
plt.rcParams['figure.figsize'] = [15, 15]

In [3]:
def remove_small_comps_3d(image, thresh = 500):
    """

    Parameters
    ----------
    image : binary np array with uint8 elements
        3d numpy matrix, connected components will be removed form this image
    thresh : int64
        smallest connected components to keep

    Returns
    -------
    np.array with uint8 elements, binary
        binary image with connected components below the threshold removed.

    """
    img_lab, N = cc3d.connected_components(image,return_N=True)
    unique, counts = np.unique(img_lab, return_counts=True)
    unique_keep = unique[counts>thresh]
    unique_keep = np.delete(unique_keep,[0])
    img_filt = np.zeros(img_lab.shape).astype('int8')
    img_filt[np.isin(img_lab,unique_keep)] = 1
    return img_filt.astype('uint8')   

def fill_holes(img,thresh=1000):
    #res = np.zeros(img.shape)
    for i in np.unique(img)[::-1]:
        _tmp = (img==i)*1.0
        _tmp = _tmp.astype('int8')
        _tmp = remove_small_comps_3d(_tmp,thresh=thresh)
        img[_tmp==1] = i
    res = img.astype('int8')
    return res

def _rotmat(vector, points):
    """
    Rotates a 3xn array of 3D coordinates from the +z normal to an
    arbitrary new normal vector.
    """
    
    vector = vg.normalize(vector)
    axis = vg.perpendicular(vg.basis.z, vector)
    angle = vg.angle(vg.basis.z, vector, units='rad')
    
    a = np.hstack((axis, (angle,)))
    R = matrix_from_axis_angle(a)
    
    r = sp.spatial.transform.Rotation.from_matrix(R)
    rotmat = r.apply(points)
    
    return rotmat

def closest_node(node, nodes):
    nodes = np.asarray(nodes)
    dist_2 = np.sum((nodes - node)**2, axis=1)
    if np.min(dist_2)>10:
        return node
    else:
        return nodes[np.argmin(dist_2)]

In [4]:
folder  = Path('../FILES_FOR_SHRUTI')
files = list(folder.glob('LSFM/*/*.tif')) #+ list(Path('../FILES_FOR_SHRUTI').glob('*/*/*_up.tif'))
files = [x.as_posix() for x in files]
files  = [x for x in files if 'reshape' not in x]

In [ ]:
files

In [ ]:
for file in files:
    img = np.expand_dims(np.swapaxes(io.imread(file), 0,2),0)/65535 * 1023
    #_ = np.zeros(img.shape)
    _ = np.random.normal(loc = np.mean(img), scale = np.std(img), size = img.shape)
    img = np.concatenate((img, _), axis = 0)
    io.imsave(re.sub('.tif','_reshape.tif', file),img)
    print(img.shape)

In [22]:
folder  = Path('../FILES_FOR_SHRUTI')
files = list(Path('../FILES_FOR_SHRUTI').glob('*/*/*_up.tif'))
files = [x.as_posix() for x in files]
files  = [x for x in files if 'reshape' not in x]
files

['../FILES_FOR_SHRUTI/ROSA/2520/C1-XYZ1_up.tif',
 '../FILES_FOR_SHRUTI/ROSA/2979/C1-XYZ1_up.tif',
 '../FILES_FOR_SHRUTI/ROSA/2981/C1-XYZ1_up.tif',
 '../FILES_FOR_SHRUTI/ROSA/4034/C1-XYZ1_up.tif',
 '../FILES_FOR_SHRUTI/ROSA/4054/C2-XYZ1_up.tif',
 '../FILES_FOR_SHRUTI/C57/04/C1-XYZres468-1_up.tif',
 '../FILES_FOR_SHRUTI/C57/03/C1-XYZres460_0001_up.tif',
 '../FILES_FOR_SHRUTI/C57/01/C1-XYZres438_up.tif',
 '../FILES_FOR_SHRUTI/C57/02/C1-XYZres445_0001_up.tif']

In [23]:
for file in files[4:]:
    print(file)
    img = np.expand_dims(np.swapaxes(io.imread(file), 0,2),0)/65535 * 1023
    #_ = np.zeros(img.shape)
    _ = np.random.normal(loc = np.mean(img), scale = np.std(img), size = img.shape)
    img = np.concatenate((img, _), axis = 0)
    io.imsave(re.sub('.tif','_reshape.tif', file),img)
    print(img.shape)
    break

../FILES_FOR_SHRUTI/ROSA/4054/C2-XYZ1_up.tif
(2, 512, 512, 507)


In [13]:
len(img.shape)

4

In [11]:
files = list(Path('../FILES_FOR_SHRUTI').glob('*/*/*_reshape.tif'))

In [12]:
files

[PosixPath('../FILES_FOR_SHRUTI/ROSA/2520/C1-XYZ1_up_reshape.tif'),
 PosixPath('../FILES_FOR_SHRUTI/ROSA/2979/C1-XYZ1_up_reshape.tif'),
 PosixPath('../FILES_FOR_SHRUTI/ROSA/2981/C1-XYZ1_up_reshape.tif'),
 PosixPath('../FILES_FOR_SHRUTI/ROSA/4034/C1-XYZ1_up_reshape.tif'),
 PosixPath('../FILES_FOR_SHRUTI/ROSA/4054/XYZ1_up_reshape.tif'),
 PosixPath('../FILES_FOR_SHRUTI/C57/04/C1-XYZres468-1_up_reshape.tif'),
 PosixPath('../FILES_FOR_SHRUTI/C57/03/C1-XYZres460_0001_up_reshape.tif'),
 PosixPath('../FILES_FOR_SHRUTI/C57/01/C1-XYZres438_up_reshape.tif'),
 PosixPath('../FILES_FOR_SHRUTI/C57/02/C1-XYZres445_0001_up_reshape.tif'),
 PosixPath('../FILES_FOR_SHRUTI/LSFM/64/64_reshape.tif'),
 PosixPath('../FILES_FOR_SHRUTI/LSFM/78A/Substack (571-669)_reshape.tif'),
 PosixPath('../FILES_FOR_SHRUTI/LSFM/69/69_reshape.tif'),
 PosixPath('../FILES_FOR_SHRUTI/LSFM/65/65_reshape.tif')]

In [14]:
files = list(Path('../FILES_FOR_SHRUTI').glob('*/*/*_reshape.tif'))#grab folder names/mouse ids
mouse_ids = sorted([x.as_posix() for x in files])
np.random.shuffle(mouse_ids)
data_dicts = [
    {"image":image_name}
    for image_name in mouse_ids if not os.path.exists(re.sub('.tif','_pred.npy',image_name))
]

data_dicts

[{'image': '../FILES_FOR_SHRUTI/C57/02/C1-XYZres445_0001_up_reshape.tif'},
 {'image': '../FILES_FOR_SHRUTI/ROSA/2979/C1-XYZ1_up_reshape.tif'},
 {'image': '../FILES_FOR_SHRUTI/C57/03/C1-XYZres460_0001_up_reshape.tif'},
 {'image': '../FILES_FOR_SHRUTI/ROSA/4034/C1-XYZ1_up_reshape.tif'},
 {'image': '../FILES_FOR_SHRUTI/C57/04/C1-XYZres468-1_up_reshape.tif'},
 {'image': '../FILES_FOR_SHRUTI/C57/01/C1-XYZres438_up_reshape.tif'},
 {'image': '../FILES_FOR_SHRUTI/ROSA/2520/C1-XYZ1_up_reshape.tif'},
 {'image': '../FILES_FOR_SHRUTI/ROSA/4054/XYZ1_up_reshape.tif'},
 {'image': '../FILES_FOR_SHRUTI/ROSA/2981/C1-XYZ1_up_reshape.tif'},
 {'image': '../FILES_FOR_SHRUTI/LSFM/78A/Substack (571-669)_reshape.tif'}]

In [71]:
files = list(Path('../FILES_FOR_SHRUTI').glob('*/*/*_reshape.tif'))#grab folder names/mouse ids
mouse_ids = sorted([x.as_posix() for x in files])
np.random.shuffle(mouse_ids)
data_dicts = [
    {"image":image_name}
    for image_name in mouse_ids if not os.path.exists(re.sub('.tif','_pred.npy',image_name))
]

print(len(data_dicts))

#data_dicts = [data_dicts[_i]]

pred_transforms = Compose(
    [
        LoadImaged(keys=["image"],reader = TIFFReader(channel_dim = 0)),
        EnsureChannelFirstd(keys=["image"]),
        Spacingd(keys=["image"], pixdim=(
            1, 1, 1), mode=("bilinear")),
        ScaleIntensityRanged(
            keys=["image"], a_min=0, a_max=1024,
            b_min=0.0, b_max=1.0, clip=True,
        ),
        EnsureTyped(keys=["image"]),
    ]
)

pred_ds = Dataset(data=data_dicts, transform=pred_transforms)
pred_loader = DataLoader(pred_ds, batch_size=1, shuffle=False)

num_evals = 20
post_pred = Compose([EnsureType(), AsDiscrete(argmax=True,to_onehot=3)])
softmax = torch.nn.Softmax(dim=1)
#model.eval()
#for m in model.modules():
#    if m.__class__.__name__.startswith('Dropout'):
#        m.train()
with torch.no_grad():
    for i, pred_data in tqdm(enumerate(pred_loader)):
        print(pred_data["image"].shape)
        _shape = pred_data["image"].shape
        print(np.empty((num_evals,3,_shape[2],_shape[3],_shape[4]),dtype=np.float16).shape)

9


1it [00:02,  2.58s/it]

torch.Size([1, 2, 1025, 742, 307])
(20, 3, 1025, 742, 307)


2it [00:22, 12.96s/it]

torch.Size([1, 2, 1025, 1025, 780])
(20, 3, 1025, 1025, 780)


...

torch.Size([1, 2, 512, 512, 253])
(20, 3, 512, 512, 253)


8it [00:50,  7.75s/it]

torch.Size([1, 2, 1025, 1025, 706])
(20, 3, 1025, 1025, 706)


9it [00:51,  5.72s/it]

torch.Size([1, 2, 512, 512, 253])
(20, 3, 512, 512, 253)


In [15]:
files = list(Path('../FILES_FOR_SHRUTI').glob('*/*/*_meante*'))

In [16]:
files

[PosixPath('../FILES_FOR_SHRUTI/ROSA/2520/C1-XYZ1_up_reshape_meante.npy'),
 PosixPath('../FILES_FOR_SHRUTI/ROSA/2981/C1-XYZ1_up_reshape_meante.npy'),
 PosixPath('../FILES_FOR_SHRUTI/ROSA/4034/C1-XYZ1_up_reshape_meante.npy'),
 PosixPath('../FILES_FOR_SHRUTI/C57/04/C1-XYZres468-1_up_reshape_meante.npy'),
 PosixPath('../FILES_FOR_SHRUTI/C57/03/C1-XYZres460_0001_up_reshape_meante.npy'),
 PosixPath('../FILES_FOR_SHRUTI/C57/01/C1-XYZres438_up_reshape_meante.npy'),
 PosixPath('../FILES_FOR_SHRUTI/C57/02/C1-XYZres445_0001_up_reshape_meante.npy'),
 PosixPath('../FILES_FOR_SHRUTI/LSFM/64/64_reshape_meante.npy'),
 PosixPath('../FILES_FOR_SHRUTI/LSFM/78A/Substack (571-669)_reshape_meante.npy'),
 PosixPath('../FILES_FOR_SHRUTI/LSFM/69/69_reshape_meante.npy'),
 PosixPath('../FILES_FOR_SHRUTI/LSFM/65/65_reshape_meante.npy')]

In [9]:
for file in tqdm(files):
    mean = np.load(file)
    seg = (mean > 0.5) * 1
    seg = np.argmax(seg, axis = 0)
    io.imsave(re.sub('_mean.npy','_seg.tif', file.as_posix()),seg)

  0%|          | 0/9 [00:00<?, ?it/s]/tmp/ipykernel_99483/2384760296.py:5: UserWarning:

../FILES_FOR_SHRUTI/ROSA/2520/C1-XYZ1_up_reshape_seg.tif is a low contrast image

 11%|█         | 1/9 [00:07<00:57,  7.18s/it]/tmp/ipykernel_99483/2384760296.py:5: UserWarning:

../FILES_FOR_SHRUTI/ROSA/2981/C1-XYZ1_up_reshape_seg.tif is a low contrast image

 22%|██▏       | 2/9 [00:15<00:56,  8.06s/it]/tmp/ipykernel_99483/2384760296.py:5: UserWarning:

../FILES_FOR_SHRUTI/ROSA/4034/C1-XYZ1_up_reshape_seg.tif is a low contrast image

 33%|███▎      | 3/9 [00:21<00:41,  6.94s/it]/tmp/ipykernel_99483/2384760296.py:5: UserWarning:

../FILES_FOR_SHRUTI/C57/03/C1-XYZres460_0001_up_reshape_seg.tif is a low contrast image

 44%|████▍     | 4/9 [00:25<00:29,  5.91s/it]/tmp/ipykernel_99483/2384760296.py:5: UserWarning:

../FILES_FOR_SHRUTI/C57/01/C1-XYZres438_up_reshape_seg.tif is a low contrast image

 56%|█████▌    | 5/9 [00:30<00:21,  5.40s/it]/tmp/ipykernel_99483/2384760296.py:5: UserWarning:

../FILE

In [10]:
re.sub('_mean.npy','_seg.tif', file.as_posix())

'../FILES_FOR_SHRUTI/LSFM/65/65_reshape_seg.tif'

In [11]:
file

PosixPath('../FILES_FOR_SHRUTI/LSFM/65/65_reshape_mean.npy')